In [3]:
import os
import numpy as np
import pandas as pd
from utils.utils import *
from utils.kabsch2D import *
from nsd_access import NSDAccess 
from scipy import stats

In [4]:
nsda = NSDAccess(nsd_dir)

In [5]:
def create_rotation_df(subj, rois, sess):
    cols = ['source', 'base', 'target', 'U', 'error']
    rotations = np.zeros(((len(rois) * (len(rois)-1)) * 2, len(cols)), dtype=object)
    i = 0 
    for roi_source in rois.keys():
        mds_source = np.load(f'/home/stan/thesis-repo/data/MDS/{subj}/{subj}_{sess}_{roi_source}_mds_betas_train.npy', allow_pickle=False)
        for roi_target in rois.keys():
            if roi_source != roi_target:
                for j in range(2):
                    mds_target = np.load(f'/home/stan/thesis-repo/data/MDS/{subj}/{subj}_{sess}_{roi_target}_mds_betas_train.npy', allow_pickle=False)
                    # flip the target when j is at 1, does nothing when j = 0 
                    rotations[i, 1] = roi_target
                    if j == 1:
                        mds_target = np.dot(mds_target, np.array([[-1, 0], [0, 1]]))
                        roi_target = roi_target + "_flipped"
                    U, t= kabsch2D(mds_source, mds_target, translate=True )
                    rotations[i, 0] = roi_source
                    rotations[i, 2] = roi_target
                    rotations[i, 3] = U
                    rotated_source = rotate(mds_source, U)
                    rotations[i, 4] = error(rotated_source, mds_target, t)
                    i += 1
    df = pd.DataFrame(rotations, columns=cols)
    df = df.astype({'error': 'float32'}) # needed for later indexing
    return df

In [6]:
def get_ranking(df):
    # get the index of the lowest error for each pair in each ROI
    indx_to_keep = df.groupby(['source', 'base'])['error'].idxmin()
    df_filtered = df.loc[indx_to_keep]
    rank = df_filtered.groupby('source')['error'].mean().sort_values()
    return rank

https://www.sciencedirect.com/science/article/pii/S0167715298000066
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.weightedtau.html

Its cool but doesn't give a p value

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html#scipy.stats.kendalltau 
Normal Kendall Tau might be the thing?? 

I asked chatGPT, and it tells me to use Friedman test, by looking at the difference between the average rank

In [7]:
rois

{'V1': 1,
 'V2': 2,
 'V3': 3,
 'hV4': 4,
 'VO-1': 5,
 'VO-2': 6,
 'PHC-1': 7,
 'PHC-2': 8,
 'LO-1': 9,
 'LO-2': 10,
 'TO-1': 11,
 'TO-2': 12}

In [8]:
def get_rank_dict(subj_list, rois, sessions):
    dict, dict_values, dict_matrices = {}, {}, {}
    for i in range(len(subj_list)):
        rotations_df = create_rotation_df(subj=subj_list[i], rois=rois, sess=sessions[i])
        rank = get_ranking(rotations_df)
        rank_int = [rois[roi] for roi in rank.index]
        dict[subj_list[i]] = rank_int
        dict_values[subj_list[i]] = rank

    return dict, dict_values

In [9]:
subj_list = [f'subj0{i}' for i in range(1,9)]
sessions = [37, 37, 29, 27, 37, 29, 37, 27]

rank_dict, rank_dict_values = get_rank_dict(subj_list=subj_list, rois=rois, sessions=sessions)
rank_dict

{'subj01': [10, 5, 7, 6, 9, 8, 11, 3, 12, 2, 4, 1],
 'subj02': [8, 6, 7, 5, 11, 10, 9, 3, 2, 12, 4, 1],
 'subj03': [6, 3, 7, 4, 11, 8, 2, 5, 9, 1, 10, 12],
 'subj04': [5, 6, 7, 8, 11, 9, 4, 10, 3, 2, 12, 1],
 'subj05': [7, 5, 6, 8, 9, 10, 3, 4, 11, 12, 2, 1],
 'subj06': [7, 8, 3, 10, 5, 9, 6, 2, 11, 12, 4, 1],
 'subj07': [3, 2, 6, 1, 4, 5, 7, 9, 11, 10, 12, 8],
 'subj08': [3, 7, 6, 2, 4, 9, 5, 10, 1, 8, 11, 12]}

In [16]:
rank_dict_values.values()

dict_values([source
LO-2     5409.333984
VO-1     5858.212891
PHC-1    6010.904785
VO-2     6028.265137
LO-1     6075.488770
PHC-2    6169.129395
TO-1     6525.177734
V3       6543.895508
TO-2     6574.104492
V2       6597.642578
hV4      6777.357910
V1       7091.245117
Name: error, dtype: float32, source
PHC-2    5157.920898
VO-2     5211.248047
PHC-1    5353.651855
VO-1     5366.352051
TO-1     5469.051758
LO-2     5599.638672
LO-1     5952.505371
V3       6206.214355
V2       6266.370605
TO-2     6489.022949
hV4      6832.226074
V1       6856.600098
Name: error, dtype: float32, source
VO-2     4454.354004
V3       4837.339844
PHC-1    4899.449707
hV4      4991.247070
TO-1     5136.125977
PHC-2    5213.922363
V2       5269.404297
VO-1     5335.218262
LO-1     5465.896973
V1       5607.233887
LO-2     5888.552734
TO-2     6229.059570
Name: error, dtype: float32, source
VO-1     4290.905273
VO-2     4553.198730
PHC-1    4741.291504
PHC-2    4809.387207
TO-1     4908.246582
LO-1     51

In [73]:
res = stats.weightedtau(rank_01_int, rank_02_int, additive=False)

In [107]:
corr = np.zeros((len(subj_list), len(subj_list)))
p_values = np.zeros((len(subj_list), len(subj_list)))

for i in range(len(subj_list)):
    for j in range(len(subj_list)):
        res = stats.kendalltau(list(rank_dict.values())[i], list(rank_dict.values())[j])
        corr[i, j] = res.correlation
        p_values[i, j] = round(res.pvalue, 5)
        

In [108]:
corr_df = pd.DataFrame(corr, columns=rank_dict.keys(), index=rank_dict.keys())
corr_df

,subj01,subj02,subj03,subj04,subj05,subj06,subj07,subj08
subj01,1.000000,0.272727,0.060606,-0.030303,0.303030,0.272727,-0.121212,-0.606061
subj02,0.272727,1.000000,-0.303030,0.030303,0.424242,0.272727,-0.121212,-0.121212
subj03,0.060606,-0.303030,1.000000,0.242424,-0.090909,-0.303030,0.151515,0.151515
subj04,-0.030303,0.030303,0.242424,1.000000,-0.060606,-0.212121,-0.121212,0.121212
subj05,0.303030,0.424242,-0.090909,-0.060606,1.000000,0.666667,-0.090909,-0.454545
subj06,0.272727,0.272727,-0.303030,-0.212121,0.666667,1.000000,-0.181818,-0.424242
subj07,-0.121212,-0.121212,0.151515,-0.121212,-0.090909,-0.181818,1.000000,0.333333
subj08,-0.606061,-0.121212,0.151515,0.121212,-0.454545,-0.424242,0.333333,1.000000


In [109]:
pvalues_df = pd.DataFrame(p_values, columns=rank_dict.keys(), index=rank_dict.keys())
pvalues_df

,subj01,subj02,subj03,subj04,subj05,subj06,subj07,subj08
subj01,0.00000,0.24958,0.84059,0.94656,0.19695,0.24958,0.63836,0.00538
subj02,0.24958,0.00000,0.19695,0.94656,0.06287,0.24958,0.63836,0.63836
subj03,0.84059,0.19695,0.00000,0.31081,0.73731,0.19695,0.54520,0.54520
subj04,0.94656,0.94656,0.31081,0.00000,0.84059,0.38070,0.63836,0.63836
subj05,0.19695,0.06287,0.73731,0.84059,0.00000,0.00180,0.73731,0.04474
subj06,0.24958,0.24958,0.19695,0.38070,0.00180,0.00000,0.45902,0.06287
subj07,0.63836,0.63836,0.54520,0.63836,0.73731,0.45902,0.00000,0.15259
subj08,0.00538,0.63836,0.54520,0.63836,0.04474,0.06287,0.15259,0.00000


In [125]:
rank_total = np.zeros(len(rois))
for i in range(len(subj_list)):
    rank_total += list(rank_dict_values.values())[i]

In [127]:
rank_total.sort_values()

source
VO-2     42259.394043
PHC-1    42331.532227
VO-1     43387.221680
V3       44794.412109
PHC-2    45138.485840
LO-1     45486.438477
LO-2     45651.568848
TO-1     46509.054199
V2       46907.472168
hV4      47306.250977
V1       50371.551270
TO-2     50883.858398
Name: error, dtype: float64

In [22]:
rank_dict_values_norm = {key : (value - value.min()) / (value.max() - value.min()) for key, value in rank_dict_values.items()}
rank_dict_values_norm

{'subj01': source
 LO-2     0.000000
 VO-1     0.266886
 PHC-1    0.357671
 VO-2     0.367993
 LO-1     0.396070
 PHC-2    0.451745
 TO-1     0.663438
 V3       0.674567
 TO-2     0.692528
 V2       0.706523
 hV4      0.813375
 V1       1.000000
 Name: error, dtype: float32,
 'subj02': source
 PHC-2    0.000000
 VO-2     0.031393
 PHC-1    0.115225
 VO-1     0.122702
 TO-1     0.183160
 LO-2     0.260036
 LO-1     0.467766
 V3       0.617123
 V2       0.652536
 TO-2     0.783610
 hV4      0.985651
 V1       1.000000
 Name: error, dtype: float32,
 'subj03': source
 VO-2     0.000000
 V3       0.215802
 PHC-1    0.250800
 hV4      0.302525
 TO-1     0.384161
 PHC-2    0.427997
 V2       0.459259
 VO-1     0.496344
 LO-1     0.569978
 V1       0.649618
 LO-2     0.808133
 TO-2     1.000000
 Name: error, dtype: float32,
 'subj04': source
 VO-1     0.000000
 VO-2     0.171322
 PHC-1    0.294178
 PHC-2    0.338656
 TO-1     0.403228
 LO-1     0.570510
 hV4      0.610859
 LO-2     0.625919
 V

In [25]:
rank_total_norm = np.zeros(len(rois))
for i in range(len(subj_list)):
    rank_total_norm += list(rank_dict_values_norm.values())[i]

rank_total_norm.sort_values()

# Def VO-2

source
VO-2     1.407972
PHC-1    1.449886
VO-1     2.156212
V3       2.893096
PHC-2    3.283835
LO-1     3.414305
LO-2     3.550135
TO-1     4.146005
V2       4.167119
hV4      4.535927
V1       6.408446
TO-2     6.848409
Name: error, dtype: float64

In [147]:
# apply rotation on one subject
def apply_rotation(target, subj, rois, sessions):
    # obtain the rotation matrix again
    rotations_df = create_rotation_df(subj, rois, sessions)
    rotations_df = rotations_df[rotations_df['base'] == target]
    inxd_to_keep = rotations_df.groupby(['source', 'base'])['error'].idxmin()
    rotations_df = rotations_df.loc[indx_to_keep]
    rotations_df = rotations_df.reset_index(drop=True)
    print(rotations_df.U)
    rotated_path = os.path.join(proj_dir, 'MDS_rotated', subj)
    print(rotated_path)
    if not os.path.exists(rotated_path):
        os.makedirs(rotated_path)
        print(os.path.exists(rotated_path))
    for roi in rois.keys():
        source_mds = np.load(f'/home/stan/thesis-repo/data/MDS/{subj}/{subj}_{sessions}_{roi}_mds_betas_train.npy', allow_pickle=False)
        mds_rotated_file = os.path.join(rotated_path, f'{subj}_{sessions}_{roi}_mds_betas_train_rotated.npy')
        # if the source is our reference point, just save it without applying rotation
        if roi == target:
            if not os.path.exists(mds_rotated_file):
                np.save(mds_rotated_file, source_mds, allow_pickle=True)
            continue 
        U = rotations_df.loc[rotations_df['source'] == roi, 'U'].squeeze()
        mds_rotated = np.dot(source_mds, U)  # apply rotation with dot product
        if not os.path.exists(mds_rotated_file):
            np.save(mds_rotated_file, mds_rotated, allow_pickle=True)
    
    

In [150]:
for i in range(1, 9):
    apply_rotation('VO-2', f'subj0{i}', rois, sessions[i-1])


0     [[-0.8724778271944559, 0.4886537025891044], [-...
1     [[0.96454704, -0.26391086], [0.26391086, 0.964...
2     [[-0.6130681308322099, -0.7900300418072094], [...
3     [[-0.40568583696046456, -0.9140125828946161], ...
4     [[-0.9887476130446997, -0.14959330766584739], ...
5     [[0.24668712449818908, 0.9690951772694021], [-...
6     [[-0.66655576, -0.74545515], [0.74545515, -0.6...
7     [[-0.9893124113883656, 0.14581135988988306], [...
8     [[-0.7697200120262242, 0.6383816280927495], [-...
9     [[-0.908599, -0.41766948], [0.41766948, -0.908...
10    [[0.9996944, -0.024719536], [0.024719536, 0.99...
Name: U, dtype: object
/home/stan/thesis-repo/projects/NSD/MDS_rotated/subj01
0     [[0.7259119404769426, 0.6877876523121068], [-0...
1     [[0.6764271, 0.73650956], [-0.73650956, 0.6764...
2     [[-0.9758575224487703, 0.2184080948136929], [-...
3     [[-0.9771164281430665, -0.21270516179194204], ...
4     [[-0.9997328567303292, -0.023113095288498975],...
5     [[-0.574897453313931